# PSTAT 134
## Assignment 3
## Spring 2018
## J Steven Raquel

###  Problem 1



In [ ]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import subprocess as sp
import pickle
import sklearn.decomposition as skld

import helper_basketball as h
import imp
imp.reload(h);

In [ ]:
allshots = pickle.load(open('allshots2016-17.pkl', 'rb'))
X = pickle.load(open('allpatterns2016-17.pkl', 'rb'))

# allmade = allshots.loc[allshots.SHOT_MADE_FLAG==1]
allmade = allshots
allmade.head()

In [ ]:
## get all 2016-17 teams
params = {'LeagueID':'00','Season': '2016-17'}
teams = h.get_nba_data('commonTeamYears', params).set_index('TEAM_ID')
allteams = teams.loc[teams.MAX_YEAR=='2017'].index.values


## get all 2016-17 players
params = {'LeagueID':'00', 'Season': '2016-17', 'IsOnlyCurrentSeason': '0'}
players = h.get_nba_data('commonallplayers', params).set_index('PERSON_ID')
allplyrs = players.loc[players.TEAM_ID.isin(allteams)].index.values

In [ ]:
## players info
player_ids = allmade.PlayerID.unique()
num_players = player_ids.size

## bin edge definitions in inches
xedges = (np.linspace(start=-25, stop=25, num=151, dtype=np.float)) * 12
yedges = (np.linspace(start= -4, stop=31, num=106, dtype=np.float)) * 12

## number of bins is one less than number of edges
nx = xedges.size - 1
ny = yedges.size - 1

## 2d histogram containers for binned counts and smoothed binned counts
all_counts = {}
all_smooth = {}

## data matrix: players (row) by vectorized 2-d court locations (column)
for i, one in enumerate(allmade.groupby('PlayerID')):
    
    ## what does this line do?
    pid, pdf = one
    
    ## h.bin_shots: what is this function doing?
    tmp1, xedges, yedges = h.bin_shots(pdf, bin_edges=(xedges, yedges), density=True, sigma=2)
    tmp2, xedges, yedges = h.bin_shots(pdf, bin_edges=(xedges, yedges), density=False)
    
    ## vectorize and store into dictionary
    all_smooth[pid] = tmp1.reshape(-1)
    all_counts[pid] = tmp2.reshape(-1)

## Problem 1



In [ ]:
## Non-negative Matrix Factorization
# X = np.stack(all_smooth.values()).T

## what are the different options mean for NMF()?
# 10 components
model = skld.NMF(n_components=10, init='nndsvda', max_iter=500, random_state=0)
W = model.fit_transform(X)
H = model.components_

# visulizing the bases with
fig, ax = plt.subplots(nrows = 5, ncols = 2, figsize=(20,40))

for i, axi in enumerate(ax.flatten()):
    h.plot_shotchart(W[:,i], xedges, yedges, ax=axi)
    axi.set_title('NMF component ' + str(i))

In [ ]:
## what are the different options mean for NMF()?
# 5 components
model = skld.NMF(n_components=5, init='nndsvda', max_iter=500, random_state=0)
W = model.fit_transform(X)
H = model.components_

# visualizing the bases
fig, ax = plt.subplots(nrows = 5, ncols = 1, figsize=(20,40))

for i, axi in enumerate(ax.flatten()):
    h.plot_shotchart(W[:,i], xedges, yedges, ax=axi)
    axi.set_title('NMF component ' + str(i))
    
plt.tight_layout()

In [ ]:
## what are the different options mean for NMF()?
# 12 components
model = skld.NMF(n_components=12, init='nndsvda', max_iter=500, random_state=0)
W = model.fit_transform(X)
H = model.components_

# visualizing the bases 
fig, ax = plt.subplots(nrows = 6, ncols = 2, figsize=(20,40))

for i, axi in enumerate(ax.flatten()):
    h.plot_shotchart(W[:,i], xedges, yedges, ax=axi)
    axi.set_title('NMF component ' + str(i))
    
plt.tight_layout()

It appears that the value of $r$ that is too big is $12$, since some of the bases appear to be duplicated, namely the ones whose 'hot zones' are very close to the hoop. They all look very alike. Conversely there are shooting styles that are not represented when using only $5$ bases, whereas with $10$ bases it appears that all are represented without duplication. 

## Problem 2

It is of interest to look at the pairwise correlation between shooting patterns of different players. 

In [ ]:
X = np.stack(all_smooth.values()).T

## what are the different options mean for NMF()?
# 10 components
model = skld.NMF(n_components=10, init='nndsvda', max_iter=500, random_state=0)
W = model.fit_transform(X)
H = model.components_

In [ ]:
p, n = X.shape

print('Number of bins (p)   :', p)
print('Number of players (n):', n)

p_w, r = W.shape

print('Number of bins (p) :', p_w)
print('Number of bases (r):', r)

r_h, n_h = H.shape

print('Number of bases (r)  :', r_h)
print('Number of players (n):', n_h)

In [ ]:
# Getting all 362 players' shooting styles
ids = players.loc[player_ids].dropna()
Hd = pd.DataFrame(H, columns=all_smooth.keys())
coeff = Hd.loc[:,ids.index.values]
coeff.columns = ids.DISPLAY_FIRST_LAST

# scaling coefficients to sum to 1
coeff.T.sum(1)
coeff /= coeff.sum(0)
# transposing so names are in the rows
R = Hd.corr()
R.shape

We see thusly that the shape of the correlation matrix $R$ is $(362,362)$, since we are comparing each of the 362 players to one another.

Now let's visualize the matrix $R$ with the `heatmap` function from `seaborn`.

In [ ]:
sb.heatmap(R)

From the looks of it, the two players that have the highest similarities are those represented by 1 and 2, who are Stephen Curry and Marc Gasol, respectively.

In [ ]:
## create figure and axes
fig, ax = plt.subplots(2, 2, figsize=(20,40))

# 201939 = Curry, 201188 = Gasol
for axi, plyri in enumerate([201939,201188]):
    
    h.plot_shotchart(all_counts[plyri], xedges, yedges, ax=ax[axi,0])
    h.plot_shotchart(all_smooth[plyri], xedges, yedges, ax=ax[axi,1])
    
    ax[axi,0].set_title(players.DISPLAY_FIRST_LAST[plyri]+', '+str(all_counts[plyri].sum().astype('int')))
    ax[axi,1].set_title(players.DISPLAY_FIRST_LAST[plyri]+', '+str(all_counts[plyri].sum().astype('int')))
    
plt.tight_layout()

On the other hand it seems that the most dissimilar players are Stephen Curry and Brook Lopez.

In [ ]:
## create figure and axes
fig, ax = plt.subplots(2, 2, figsize=(20,40))

# 201939 = Curry, 201572 = Gasol
for axi, plyri in enumerate([201939,201572]):
    
    h.plot_shotchart(all_counts[plyri], xedges, yedges, ax=ax[axi,0])
    h.plot_shotchart(all_smooth[plyri], xedges, yedges, ax=ax[axi,1])
    
    ax[axi,0].set_title(players.DISPLAY_FIRST_LAST[plyri]+', '+str(all_counts[plyri].sum().astype('int')))
    ax[axi,1].set_title(players.DISPLAY_FIRST_LAST[plyri]+', '+str(all_counts[plyri].sum().astype('int')))
    
plt.tight_layout()